# **Знакомство с данными: винные обзоры**

In [32]:
import pandas as pd
from IPython.display import display
import math

**[После просмотра документального фильма о сомелье вы захотели создать прогностическую модель для оценки вин вслепую, как это делает сомелье.]**

Определив бизнес-задачу, вы перешли к сбору данных для обучения модели. После нескольких недель парсинга сайта [WineEnthusiast](https://www.wineenthusiast.com/) вам удалось собрать около 130 тысяч строк обзоров вин для анализа и обучения.

Вот какие признаки вам удалось собрать:

* country — страна-производитель вина.
* description — подробное описание.
* designation — название виноградника, где выращивают виноград для вина.
* points — баллы, которыми WineEnthusiast оценил вино по шкале от 1 до 100.
* price — стоимость бутылки вина.
* province — провинция или штат.
* region_1 — винодельческий район в провинции или штате (например Напа).
* region_2 — конкретный регион. Иногда в пределах винодельческой зоны указываются более конкретные регионы (например Резерфорд в долине Напа), но это значение может быть пустым.
* taster_name — имя сомелье.
* taster_twitter_handle — твиттер сомелье.
* title — название вина, которое часто содержит год и другую подробную информацию.
* variety — сорт винограда, из которого изготовлено вино (например Пино Нуар).
* winery — винодельня, которая производила вино.

In [40]:
data = pd.read_csv('data/wine.csv')
#display(data.head())

# Сколько всего дегустаторов приняло участие в винных обзорах?
display(data['taster_name'].nunique())

# Какова максимальная цена за бутылку в этом наборе данных? Введите целое число.
display(data['price'].max())

19

3300.0

In [24]:
print(data.columns[1])

description


In [29]:
# Проанализируйте представленный датасет и перечислите все числовые признаки через запятую.
display(data.info())
int_float_cols = []
i = 0

for type in data.dtypes:
    if type in ['int64', 'float64']:
        int_float_cols.append(data.columns[i])
    i+=1
display(int_float_cols)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


None

['points', 'price']

In [30]:
# Проанализируйте датасет на наличие дублирующихся винных обзоров. Если дублирующиеся записи есть, удалите их.
# В поле ниже введите, сколько дублирующихся записей вам удалось обнаружить.

dupl_columns = list(data.columns)

mask = data.duplicated(subset=dupl_columns) # Создадим маску дубликатов с помощью метода duplicated() и произведём фильтрацию
duplicates = data[mask]
print(f'Число найденных дубликатов: {duplicates.shape[0]}')

dedupped = data.drop_duplicates(subset=dupl_columns) # Создадим новую таблицу sber_dedupped, которая будет версией исходной таблицы, очищенной от полных дубликатов.
print(f'Результирующее число записей: {dedupped.shape[0]}')

Число найденных дубликатов: 9983
Результирующее число записей: 119988


In [31]:
# Проверьте датасет на наличие пропусков в данных.
# В каких из представленных признаках были обнаружены пропуски?
cols_null_percent = data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

region_2                 61.136715
designation              28.825661
taster_twitter_handle    24.015357
taster_name              20.192197
region_1                 16.347493
price                     6.921544
country                   0.048472
province                  0.048472
variety                   0.000769
dtype: float64

In [42]:
# Если какой-то из признаков имеет более 30-40 % пропусков, лучше избавьтесь от него:
# его заполнение может привести к сильному искажению общего распределения, а удаление записей — к большой утрате данных.
# Заполняйте данные с умом! Если перед вами количественный признак,
# то использование нецелого числа в качестве константы является как минимум нелогичным.
# Вы можете оставить пропуски как есть, просто заменив их на какой-то специальный символ.
# Например, для числовых неотрицательных признаков можно использовать число -1, а для категориальных — строку 'unknown'.

# отбрасываем столбцы с числом пропусков более 30% (100-70)
n = dedupped.shape[0] #число строк в таблице
thresh = math.ceil(n*0.7)
data_filled = dedupped.dropna(how='any', thresh=thresh, axis=1)


# отбрасываем строки с числом пропусков более 4 в строке
m = data_filled.shape[1] #число признаков после удаления столбцов
data_filled = data_filled.dropna(how='any', thresh=m-4, axis=0)

#заполняем оставшиеся записи константами в соответствии со словарем values
data_filled=data_filled.fillna(value={'country': 'unknown', 'designation': 'unknown', 'price': -1, 'province': 'unknown', 'region_1': 'unknown', 'taster_name': 'unknown', 'taster_twitter_handle': 'unknown'}) 
data_filled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119987 entries, 0 to 129970
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                119987 non-null  object 
 1   description            119987 non-null  object 
 2   designation            119987 non-null  object 
 3   points                 119987 non-null  int64  
 4   price                  119987 non-null  float64
 5   province               119987 non-null  object 
 6   region_1               119987 non-null  object 
 7   taster_name            119987 non-null  object 
 8   taster_twitter_handle  119987 non-null  object 
 9   title                  119987 non-null  object 
 10  variety                119987 non-null  object 
 11  winery                 119987 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 11.9+ MB
